In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pyreadstat as st
import numpy as np
import matplotlib.pyplot as plt


path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT09] Econometría de Datos de Panel\Clases prácticas\PS 3-20250531\data\mod_abdata.dta"

df, meta = st.read_dta(path)
df.head(1)

,c1,ind,year,emp,wage,cap,indoutpt,n,w,k,ys,rec,yearm1,id,nL1,nL2,wL1,kL1,kL2,ysL1,ysL2,yr1977,yr1978,yr1979,yr1980,yr1981,yr1982
0,1-1,7.0,1977.0,5.041,13.1516,0.5894,95.707199,1.617604,2.576543,-0.52865,4.561294,1.0,1977.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   c1        828 non-null    object 
 1   ind       828 non-null    float64
 2   year      828 non-null    float64
 3   emp       828 non-null    float64
 4   wage      828 non-null    float64
 5   cap       828 non-null    float64
 6   indoutpt  828 non-null    float64
 7   n         828 non-null    float64
 8   w         828 non-null    float64
 9   k         828 non-null    float64
 10  ys        828 non-null    float64
 11  rec       828 non-null    float64
 12  yearm1    828 non-null    float64
 13  id        828 non-null    float64
 14  nL1       770 non-null    float64
 15  nL2       632 non-null    float64
 16  wL1       770 non-null    float64
 17  kL1       770 non-null    float64
 18  kL2       632 non-null    float64
 19  ysL1      770 non-null    float64
 20  ysL2      632 non-null    float6

In [5]:
df_reg = df.sort_values(['id', 'year'])
df_reg.dropna(subset=['n', 'nL1'], inplace=True)

import statsmodels.api as sm

# Variables
X = sm.add_constant(df_reg['nL1'])
y = df_reg['n']

# Estimar el modelo
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      n   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 7.654e+04
Date:                Sat, 31 May 2025   Prob (F-statistic):               0.00
Time:                        21:05:54   Log-Likelihood:                 450.02
No. Observations:                 770   AIC:                            -896.0
Df Residuals:                     768   BIC:                            -886.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0379      0.006     -5.948      0.0

In [8]:
from statsmodels.iolib.summary2 import summary_col

# Imprimir estilo compacto
print(summary_col([model], stars=True, model_names=['t1'], info_dict={'R-squared': lambda x: f"{x.rsquared:.3f}"}))


                   t1    
-------------------------
const          -0.0379***
               (0.0064)  
nL1            0.9967*** 
               (0.0036)  
R-squared      0.9901    
R-squared Adj. 0.9901    
R-squared      0.990     
Standard errors in
parentheses.
* p<.1, ** p<.05,
***p<.01
